# MPSGE.jl

## David Anthoff, UC Berkeley, anthoff@berkeley.edu

This is the demo that I gave during my talk at the WiNDC Annual Meeting 2021. Please note that the software I show here is very much work in progress, so things will likely change significantly going forward.

First we load the MPSGE.jl package.

In [1]:
using MPSGE

Then we create a new model and store it in the global variable `m`:

In [2]:
m = Model()

MPSGE model with 0 sectors, 0 commodities and 0 consumers.


Now we start to add things to this new model. The first addition is a parameter named `endow`, with a value of `1`:

In [3]:
@parameter(m, endow, 1.0)

MPSGE._ParameterRef{Model}(MPSGE model with 0 sectors, 0 commodities and 0 consumers.
, 1)

Then we add three sectors to the model:

In [4]:
@sector(m, X)
@sector(m, Y)
@sector(m, U)

MPSGE._SectorRef{Model}(MPSGE model with 3 sectors, 0 commodities and 0 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
, 3)

And five commodities:

In [5]:
@commodity(m, PX)
@commodity(m, PY)
@commodity(m, PU)
@commodity(m, PL)
@commodity(m, PK)

MPSGE._CommodityRef{Model}(MPSGE model with 3 sectors, 5 commodities and 0 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
, 5)

Next, we specify one consumer, including a benchmark value:

In [6]:
@consumer(m, RA, benchmark=150.)

MPSGE._ConsumerRef{Model}(MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
, 1)

Then we add three production functions:

In [7]:
@production(m, X, 1, PX, 100, [Input(PL, 50), Input(PK, 50)])
@production(m, Y, 1, PY, 50, [Input(PL, 20), Input(PK, 30)])
@production(m, U, 1, PU, 150, [Input(PX, 100), Input(PY, 50)])

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)


And finally, we add one demand function:

In [8]:
@demand(m, RA, PU, [Endowment(PL, :(70 * $endow)), Endowment(PK, 80.)])

MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)


At this point we can view a nice algebraic version of our model, just to check what we have:

In [9]:
algebraic_version(m)

Mixed complementarity problem with 15 constraints:
  (50.0 * (PL ^ (50.0 / 100.0) * PK ^ (50.0 / 100.0))) / PL - PLPX    ┴  PLPX
  (50.0 * (PL ^ (50.0 / 100.0) * PK ^ (50.0 / 100.0))) / PK - PKPX    ┴  PKPX
  (20.0 * (PL ^ (20.0 / 50.0) * PK ^ (30.0 / 50.0))) / PL - PLPY      ┴  PLPY
  (30.0 * (PL ^ (20.0 / 50.0) * PK ^ (30.0 / 50.0))) / PK - PKPY      ┴  PKPY
  (100.0 * (PX ^ (100.0 / 150.0) * PY ^ (50.0 / 150.0))) / PX - PXPU  ┴  PXPU
  (50.0 * (PX ^ (100.0 / 150.0) * PY ^ (50.0 / 150.0))) / PY - PYPU   ┴  PYPU
  (PL * PLPX + PK * PKPX) - 100.0 * PX                                ┴  0.0 < X
  (PL * PLPY + PK * PKPY) - 50.0 * PY                                 ┴  0.0 < Y
  (PX * PXPU + PY * PYPU) - 150.0 * PU                                ┴  0.0 < U
  (+(PXPU * U) + 0.0) - 100.0 * X                                     ┴  0.001 < PX
  (+(PYPU * U) + 0.0) - 50.0 * Y                                      ┴  0.001 < PY
  (0.0 + +(RA / PU)) - 150.0 * U                                      

As a first step, we will check whether the benchmark values we provied are actually a model solution:

In [10]:
solve!(m, cumulative_iteration_limit=0)

Reading options file C:\Users\david\AppData\Local\Temp\jl_3985.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (F_X)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.063000
Residual. . . . . . . . 0.000000e+00


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.0
  Y:	1.0
  U:	1.0
  PX:	1.0
  PY:	1.0
  PU:	1.0
  PL:	1.0
  PK:	1.0
  PLPX:	50.0
  PKPX:	50.0
  PLPY:	20.0
  PKPY:	30.0
  PXPU:	100.0
  PYPU:	50.0
  RA:	150.0


Next we change the value of the parameter of our model, and then resolve the model:

In [11]:
set_value(endow, 1.1)
solve!(m)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4828e+01             0.0e+00 (F_PL)
    1     1     0    15 6.0442e-01  1.0e+00    0.0e+00 (F_PU)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 6.0442e-01           I 0.0e+00 4.1e-01 (F_PU)
    1     1     3     3 3.7495e-04  1.0e+00 SO 0.0e+00 3.0e-04 (F_PK)
    2     1     4     4 1.9273e-08  1.0e+00 SO 0.0e+00 1.3e-08 (F_PLPX)

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient Evaluations. . 4
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 1.927298e-08


MPSGE model with 3 sectors, 5 commodities and 1 consumers.
  Sectors: X (bm=1.0), Y (bm=1.0), U (bm=1.0)
  Commodities: PX (bm=1.0), PY (bm=1.0), PU (bm=1.0), PL (bm=1.0), PK (bm=1.0)
  Consumers: RA (bm=150.0)
Solution:
  X:	1.048808848173496
  Y:	1.038860118257079
  U:	1.0454820635822557
  PX:	1.048856610774173
  PY:	1.0589010729788217
  PU:	1.0521941333870226
  PL:	1.000045539613954
  PK:	1.1000500941011886
  PLPX:	52.44044240818331
  PKPX:	47.67312946227609
  PLPY:	21.177057058262655
  PKPY:	28.877805079697776
  PXPU:	100.31820580255861
  PYPU:	49.68330660298151
  RA:	165.00751407836955
